## Load python libraries

In [28]:
import configparser
import gzip
import os
import glob
import subprocess
import multiprocessing
import shutil
import pandas as pd

## Import information from configuration file

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

path_data = config.get('paths', 'path_data')

print(path_data)

../data


In [3]:
def find_unique_directory_paths():
    fastq_dir = set()
    file_paths = glob.glob(path_data + '/**/*.fastq.gz', recursive=True)
    directory_paths = map(os.path.dirname, file_paths)
    fastq_dir.update(directory_paths)
    return sorted(fastq_dir)

fastq_dir = find_unique_directory_paths()
fastq_dir

['../data/barcode01', '../data/barcode02', '../data/barcode03']

In [58]:
basename = lambda paths: [path.split('/')[-2] if path.endswith('/') else path.split('/')[-1] for path in paths]
base_name = basename(fastq_dir)
print(base_name)

['barcode01', 'barcode02', 'barcode03']


## Count number of sequences per sample

In [5]:
def count_sequences(file_path):
    count = 0
    with gzip.open(file_path, 'rt') as gz_file:
        for line in gz_file:
            if line.startswith('@'):
                count += 1
    return count

def accumulate_counts(folder_path):
    counts = {}
    for folder in folder_path:
        counts[folder] = 0  # Initialize the folder key with count 0
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            if file_name.endswith('.fastq.gz'):
                counts[folder] += count_sequences(file_path)
    return counts

In [6]:
x = accumulate_counts(fastq_dir)
x

{'../data/barcode01': 28513,
 '../data/barcode02': 28554,
 '../data/barcode03': 28469}

## Filtering with NanoFilt

In [7]:
def filtering(folder_list, minimum_length, maximum_length, qscore, base_list):
    for folder, base in zip(folder_list, base_list):
        os.makedirs(os.path.join('results/qc', base))
        for file_name in os.listdir(folder):
            if file_name.endswith('.fastq.gz'):
                file_path = os.path.join(folder, file_name)
                file_name_unzipped = file_name[:-3]
                command = f'gunzip -c {file_path} | NanoFilt \
                          --length {minimum_length} \
                          --maxlength {maximum_length} \
                          -q {qscore} | gzip > ./results/qc/{base}/{file_name}'
                subprocess.run(command, shell=True)

In [8]:
filtering(fastq_dir, config.get('NanoFilt', 'minlength'), config.get('NanoFilt', 'maxlength'), config.get('NanoFilt', 'qscore'), base_name)

## Concatenate filtered files to one file per sample

In [9]:
def concatenate(bases):
    for base in bases:
        command = f'cat ./results/qc/{base}/*.fastq.gz > ./results/qc/{base}/{base}_concatenated.fastq.gz'
        subprocess.run(command, shell=True)
        
        command = f'gunzip ./results/qc/{base}/{base}_concatenated.fastq.gz'
        subprocess.run(command, shell=True)
        
        command = f'sed -n "1~4s/^@/>/p;2~4p" ./results/qc/{base}/{base}_concatenated.fastq > ./results/qc/{base}/{base}_concatenated.fasta'
        subprocess.run(command, shell=True)

        #command = f'gzip ./results/qc/{base}/{base}_concatenated.fasta'
        #subprocess.run(command, shell=True)

In [10]:
concatenate(base_name)

## Convert fasta to csv

In [62]:
for base in base_name:
    fasta = './results/qc/' + base + '/' + base + '_concatenated.fasta'
    output = './results/qc/' + base + '/' + base + '_concatenated.csv'

    out_lines = []
    temp_line = ''
    with open(fasta, 'r') as fp:
        for line in fp:
            if line.startswith('>'):
                out_lines.append(temp_line)
                temp_line = line.strip() + ','
            else:
                temp_line += line.strip()
    out_lines.append(temp_line)

    with open(output, 'w') as fp_out:
        fp_out.write('id,sequence' + '\n'.join(out_lines))

## Count sequences after filtering in concatenated fasta file

In [12]:
def count_sequences_concat(base_name):
    counts = {}
    for base in base_name:
        counts[base] = 0  # Initialize the count for each base
        file_path = './results/qc/' + base + '/' + base + '_concatenated.fasta'
        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('>'):
                    counts[base] += 1
    return counts

In [13]:
y = count_sequences_concat(base_name)
y

{'barcode01': 25040, 'barcode02': 26721, 'barcode03': 20929}

## Make sure you save the working directory before using os.chdir

In [19]:
wdir = os.getcwd()
print(wdir)

/home/pascal/Documents/git_projects/grifo/src_0.4


## Clustering

In [20]:
def process_base(base):
    os.chdir(wdir)
    shutil.copy('./ashure.py', './results/qc/' + base)
    shutil.copy('./bilge_pype.py', './results/qc/' + base)
    os.chdir('./results/qc/' + base)
    script_path = "./ashure.py"
    input_file = base + "_concatenated.csv"
    output_file = base + "_clusters.csv"
    
    command = [
        script_path,
        "clst",
        "-i", input_file,
        "-o", output_file,
        "-iter", config.get('ashure', 'niter'),
        "-r"
    ]
    
    subprocess.run(command)

if __name__ == '__main__':

    num_processes = 8  # Number of available CPU cores
    
    with multiprocessing.Pool(processes=num_processes) as pool:
        pool.map(process_base, base_name)

os.chdir(wdir)

pid[43949] 2023-06-24 09:42:03.214 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/minimap2 found
pid[43948] 2023-06-24 09:42:03.214 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/minimap2 found
pid[43947] 2023-06-24 09:42:03.214 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/minimap2 found
pid[43948] 2023-06-24 09:42:03.214 INFO: check_toolchain: /usr/bin/bwa found
pid[43947] 2023-06-24 09:42:03.214 INFO: check_toolchain: /usr/bin/bwa found
pid[43949] 2023-06-24 09:42:03.214 INFO: check_toolchain: /usr/bin/bwa found
pid[43947] 2023-06-24 09:42:03.215 INFO: check_toolchain: /usr/bin/bowtie2 found
pid[43949] 2023-06-24 09:42:03.215 INFO: check_toolchain: /usr/bin/bowtie2 found
pid[43948] 2023-06-24 09:42:03.215 INFO: check_toolchain: /usr/bin/bowtie2 found
pid[43949] 2023-06-24 09:42:03.215 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/spoa found
pid[43947] 2023-06-24 09:42:03.215 INFO: check_toolchain: /home/pascal/anaconda3/envs/

pid[43948] 2023-06-24 09:42:03.447 INFO: cluster_sample: qlen=26721 progress=0/5
pid[43948] 2023-06-24 09:42:03.450 INFO: Making directory ./clusters/


[M::mm_idx_gen::0.000*11.61] collected minimizers
[M::mm_idx_gen::0.001*5.63] sorted minimizers
[M::main::0.001*5.60] loaded/built the index for 0 target sequence(s)
[M::mm_mapopt_update::0.001*5.46] mid_occ = 1965815675
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 0
[M::mm_idx_stat::0.001*5.37] distinct minimizers: 0 (-nan% are singletons); average occurrences: -nan; average spacing: -nan
[M::worker_pipeline::0.087*2.47] mapped 20929 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.087 sec; CPU: 0.215 sec; Peak RSS: 0.116 GB


pid[43949] 2023-06-24 09:42:04.392 INFO: cluster_sample: qlen=20929 progress=1/5


[M::mm_idx_gen::0.000*10.94] collected minimizers
[M::mm_idx_gen::0.001*5.23] sorted minimizers
[M::main::0.001*5.22] loaded/built the index for 0 target sequence(s)
[M::mm_mapopt_update::0.001*5.09] mid_occ = 0
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 0
[M::mm_idx_stat::0.001*5.02] distinct minimizers: 0 (-nan% are singletons); average occurrences: -nan; average spacing: -nan
[M::worker_pipeline::0.108*2.43] mapped 25040 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.109 sec; CPU: 0.264 sec; Peak RSS: 0.120 GB
[M::mm_idx_gen::0.000*12.63] collected minimizers
[M::mm_idx_gen::0.001*5.84] sorted minimizers
[M::main::0.001*5.78] loaded/built the index for 0 target sequence(s)
[M::mm_mapopt_update::0.001*5.63] mid_occ = 0
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 0
[M::mm_idx_stat::0.001*5.55] distinct minimizers: 0 (-nan% ar

...

pid[43948] 2023-06-24 09:49:31.119 INFO: preparing precomputed data
pid[43948] 2023-06-24 09:49:31.142 INFO: cluster_compute: running optics
pid[43948] 2023-06-24 09:49:31.142 INFO: Running OPTICS
pid[43948] 2023-06-24 09:49:31.142 INFO: max_eps = 0.5
pid[43948] 2023-06-24 09:49:31.142 INFO: clust_OPTICS: iter=0 using min_samples=68
pid[43948] 2023-06-24 09:49:31.144 INFO: clust_OPTICS: clusters=1 outliers=96 delta=34.0
pid[43948] 2023-06-24 09:49:31.144 INFO: clust_OPTICS: iter=1 using min_samples=34
pid[43948] 2023-06-24 09:49:31.146 INFO: clust_OPTICS: clusters=1 outliers=71 delta=34
pid[43948] 2023-06-24 09:49:31.146 INFO: clust_OPTICS: iter=2 using min_samples=17
pid[43948] 2023-06-24 09:49:31.149 INFO: clust_OPTICS: clusters=1 outliers=22 delta=17
pid[43948] 2023-06-24 09:49:31.149 INFO: clust_OPTICS: iter=3 using min_samples=9
pid[43948] 2023-06-24 09:49:31.155 INFO: clust_OPTICS: clusters=1 outliers=8 delta=8
pid[43948] 2023-06-24 09:49:31.155 INFO: clust_OPTICS: iter=4 using m

[M::mm_idx_gen::0.003*2.71] collected minimizers
[M::mm_idx_gen::0.004*2.74] sorted minimizers
[M::main::0.004*2.74] loaded/built the index for 191 target sequence(s)
[M::mm_mapopt_update::0.005*2.59] mid_occ = 105
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 191
[M::mm_idx_stat::0.005*2.50] distinct minimizers: 12276 (84.84% are singletons); average occurrences: 1.847; average spacing: 5.463
[M::worker_pipeline::0.036*2.77] mapped 191 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D --dual=no --for-only -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.037 sec; CPU: 0.101 sec; Peak RSS: 0.230 GB


pid[43948] 2023-06-24 09:49:31.416 INFO: preparing precomputed data
pid[43948] 2023-06-24 09:49:31.449 INFO: cluster_compute: running optics
pid[43948] 2023-06-24 09:49:31.449 INFO: Running OPTICS
pid[43948] 2023-06-24 09:49:31.449 INFO: max_eps = 0.5
pid[43948] 2023-06-24 09:49:31.449 INFO: clust_OPTICS: iter=0 using min_samples=95
pid[43948] 2023-06-24 09:49:31.452 INFO: clust_OPTICS: clusters=0 outliers=190 delta=47.5
pid[43948] 2023-06-24 09:49:31.452 INFO: clust_OPTICS: iter=1 using min_samples=48
pid[43948] 2023-06-24 09:49:31.454 INFO: clust_OPTICS: clusters=1 outliers=185 delta=47
pid[43948] 2023-06-24 09:49:31.454 INFO: clust_OPTICS: iter=2 using min_samples=25
pid[43948] 2023-06-24 09:49:31.457 INFO: clust_OPTICS: clusters=2 outliers=64 delta=23
pid[43948] 2023-06-24 09:49:31.457 INFO: clust_OPTICS: iter=3 using min_samples=14
pid[43948] 2023-06-24 09:49:31.461 INFO: clust_OPTICS: clusters=1 outliers=30 delta=11
pid[43948] 2023-06-24 09:49:31.461 INFO: clust_OPTICS: iter=4 us

[M::mm_idx_gen::0.001*4.01] collected minimizers
[M::mm_idx_gen::0.002*3.59] sorted minimizers
[M::main::0.002*3.58] loaded/built the index for 55 target sequence(s)
[M::mm_mapopt_update::0.002*3.38] mid_occ = 36
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 55
[M::mm_idx_stat::0.002*3.24] distinct minimizers: 4113 (85.29% are singletons); average occurrences: 1.554; average spacing: 5.594
[M::worker_pipeline::0.009*2.84] mapped 55 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D --dual=no --for-only -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.010 sec; CPU: 0.027 sec; Peak RSS: 0.230 GB
./ashure.py:745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

pid[43948] 2023-06-24 09:49:31.632 INFO: preparing precomputed data
pid[43948] 2023-06-24 09:49:31.642 INFO: cluster_compute: running optics
pid[43948] 2023-06-24 09:49:31.642 INFO: Running OPTICS
pid[43948] 2023-06-24 09:49:31.642 INFO: max_eps = 0.5
pid[43948] 2023-06-24 09:49:31.642 INFO: clust_OPTICS: iter=0 using min_samples=27
pid[43948] 2023-06-24 09:49:31.643 INFO: clust_OPTICS: clusters=0 outliers=54 delta=13.5
pid[43948] 2023-06-24 09:49:31.643 INFO: clust_OPTICS: iter=1 using min_samples=14
pid[43948] 2023-06-24 09:49:31.645 INFO: clust_OPTICS: clusters=2 outliers=42 delta=13
pid[43948] 2023-06-24 09:49:31.645 INFO: clust_OPTICS: iter=2 using min_samples=8
pid[43948] 2023-06-24 09:49:31.647 INFO: clust_OPTICS: clusters=1 outliers=16 delta=6
pid[43948] 2023-06-24 09:49:31.647 INFO: clust_OPTICS: iter=3 using min_samples=17
pid[43948] 2023-06-24 09:49:31.648 INFO: clust_OPTICS: clusters=2 outliers=42 delta=3
pid[43948] 2023-06-24 09:49:31.648 INFO: clust_OPTICS: iter=4 using m

## Convert csv files to fasta

In [70]:
def csv2fasta(base_name):
    for base in base_name:
        csv_path = './results/qc/' + base + '/' + base + '_clusters.csv'
        output_path = './results/qc/' + base + '/' + base + '_clusters.fasta'

        if os.path.exists(csv_path):
            out_lines = []
            temp_line = ''
            with open(csv_path, 'r') as csv_file:
                for line in csv_file:
                    cols = line.split(",")
                    out_lines.append(temp_line)
                    temp_line = ">" + cols[0] + "\n" + cols[1] + "\n"

            out_lines.append(temp_line)

            with open(output_path, 'w') as csv_out:
                csv_out.write(''.join(out_lines)[13:])

In [71]:
csv2fasta(base_name)

## Run blastn

In [33]:
os.getcwd()

'/home/pascal/Documents/git_projects/grifo/src_0.4'

In [23]:
for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_clusters.fasta'
    out_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
    if os.path.exists(file_path):
        command = [
            'cutadapt',
            '-a', 'CAGCAGCCGCGGTAATTCC;max_error_rate=0.20',
            '-g', 'CCCGTGTTGAGTCAAATTAAGC;max_error_rate=0.20',
            '--revcomp',
            '-o', out_path,
            file_path
        ]
        subprocess.run(command)

This is cutadapt 3.2 with Python 3.9.7
Command line parameters: -a CAGCAGCCGCGGTAATTCC;max_error_rate=0.20 -g CCCGTGTTGAGTCAAATTAAGC;max_error_rate=0.20 --revcomp -o ./results/qc/barcode01/barcode01_clusters_cut.fasta ./results/qc/barcode01/barcode01_clusters.fasta
Processing reads on 1 core in single-end mode ...
Finished in 0.00 s (134 µs/read; 0.45 M reads/minute).

=== Summary ===

Total reads processed:                      34
Reads with adapters:                        34 (100.0%)
Reverse-complemented:                       19 (55.9%)
Reads written (passing filters):            34 (100.0%)

Total basepairs processed:        24,643 bp
Total written (filtered):         21,704 bp (88.1%)

=== Adapter 1 ===

Sequence: CAGCAGCCGCGGTAATTCC; Type: regular 3'; Length: 19; Trimmed: 1 times; Reverse-complemented: 0 times

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 100.0%
  G: 0.0%
  T: 0.0%
  none/other: 0.0%

Ove

## Taxonomic assignment using SILVA

In [36]:
path_to_blastdb = "../reference_databases/SILVA_138/SILVA_138_SSURef_tax_silva.fasta"
numthreads = 8
mts = 10
pct_ident = 90
db = 'SILVA_138'

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
    if os.path.exists(file_path):
        print("Running blastn on", base)
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'

        command = [
            "blastn",
            "-db", path_to_blastdb,
            "-query", file_path,
            "-task", "blastn",
            "-dust", "no",
            "-num_threads", str(numthreads),
            "-outfmt", "7 delim=, sseqid stitle qacc sacc evalue bitscore length pident",
            "-max_target_seqs", str(mts),
            "-perc_identity", str(pct_ident),
            "-out", output_csv
        ]

        subprocess.run(command)

Running blastn on barcode01
Running blastn on barcode02
Running blastn on barcode03


In [37]:
for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    outfile.write(line)

In [38]:
for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    print(input_csv)
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(input_csv) and os.path.getsize(input_csv) > 0:
        # load file
        df = pd.read_csv(input_csv, sep=',')

        # add column names
        df.columns=['accession', 'taxonomic_annotation', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity']

        # select only rows with alignment length >= 500 bp
        df2 = df[df['alignment_length'] >= 500]

        # arrange rows by match percentage
        df3 = df2.sort_values(by=['percentage_identity'], ascending=False)

        # keep only first row of each ASV
        df4 = df3.drop_duplicates(subset=['cluster'], keep='first', inplace=False, ignore_index=False)

        # add sample name information
        df4['#sample_name'] = base

        df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')

        df5 = df4[['#sample_name', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity', 'taxonomic_annotation']]

        df5.to_csv(output_csv, sep=';', index=False, header=False)

./results/qc/barcode01/barcode01_SILVA_138_blastn2.csv
./results/qc/barcode02/barcode02_SILVA_138_blastn2.csv
./results/qc/barcode03/barcode03_SILVA_138_blastn2.csv


/tmp/ipykernel_42819/2996724020.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['#sample_name'] = base
/tmp/ipykernel_42819/2996724020.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')
/tmp/ipykernel_42819/2996724020.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [39]:
if os.path.exists("_SILVA_138_eDNA.csv"):
    os.remove("_SILVA_138_eDNA.csv")

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(file_path):
        with open(file_path, "r") as input_file, open("_SILVA_138_eDNA.csv", "a") as output_file:
            output_file.write(input_file.read())

with open("_SILVA_138_eDNA.csv", "r") as input_file, open("SILVA_138_eDNA.csv", "w") as output_file:
    output_file.write("counts,cluster,accession,accession,evalue,bitscore,alignment_length,percentage_identity,taxonomic_annotation\n")
    for line in input_file:
        if not line.startswith("#"):
            output_file.write(line.replace(";", ",").replace("|", ","))

## Taxonomic assignment using PR2

In [35]:
path_to_blastdb = "/home/pascal/Documents/GEANS/eDNA_18S/PR2_5.0.0/pr2_version_5.0.0_SSU_taxo_long.fasta"
numthreads = 8
mts = 10
pct_ident = 90
db = 'PR2'

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
    if os.path.exists(file_path):
        print("Running blastn on", base)
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'

        command = [
            "blastn",
            "-db", path_to_blastdb,
            "-query", file_path,
            "-task", "blastn",
            "-dust", "no",
            "-num_threads", str(numthreads),
            "-outfmt", "7 delim=, sseqid stitle qacc sacc evalue bitscore length pident",
            "-max_target_seqs", str(mts),
            "-perc_identity", str(pct_ident),
            "-out", output_csv
        ]

        subprocess.run(command)

Running blastn on barcode01


Running blastn on barcode02


Running blastn on barcode03


Running blastn on barcode04


Running blastn on barcode05


Running blastn on barcode06


Running blastn on barcode07


Running blastn on barcode08
Running blastn on barcode09
Running blastn on barcode10


Running blastn on barcode11
Running blastn on barcode12


In [36]:
db = 'PR2'


for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    outfile.write(line)



In [37]:
db = 'PR2'

for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    comma_count = line.count(',')
                    if comma_count == 10:
                        comma_indices = []
                        for i, char in enumerate(line):
                            if char == ',':
                                comma_indices.append(i)

                        if len(comma_indices) >= 6:
                            line = line[:comma_indices[0]] + line[comma_indices[0]+1:]
                            line = line[:comma_indices[2]-1] + line[comma_indices[2]:]
                            line = line[:comma_indices[5]-2] + line[comma_indices[5]-1:]
                    elif comma_count == 13:
                        comma_indices = []
                        for i, char in enumerate(line):
                            if char == ',':
                                comma_indices.append(i)

                        indices_to_remove = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Create a list to track the indices to remove

                        if len(comma_indices) >= 13:
                            indices_to_remove[0] = 1  # Mark the first comma to remove
                            indices_to_remove[1] = 1  # Mark the second comma to remove
                            indices_to_remove[3] = 1  # Mark the fourth comma to remove
                            indices_to_remove[4] = 1  # Mark the fifth comma to remove
                            indices_to_remove[7] = 1  # Mark the eighth comma to remove
                            indices_to_remove[8] = 1  # Mark the ninth comma to remove

                        updated_line = ""
                        for i, char in enumerate(line):
                            if char == ',' and indices_to_remove[comma_indices.index(i)] == 1:
                                continue
                            updated_line += char

                        line = updated_line
                    outfile.write(line)

In [38]:

db = 'PR2'




for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    print(input_csv)
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(input_csv) and os.path.getsize(input_csv) > 0:
        # load file
        df = pd.read_csv(input_csv, sep=',')
    
        # add column names
        df.columns=['accession', 'taxonomic_annotation', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity']

        # select only rows with alignment length >= 500 bp
        df2 = df[df['alignment_length'] >= 500]

        # arrange rows by match percentage
        df3 = df2.sort_values(by=['percentage_identity'], ascending=False)

        # keep only first row of each ASV
        df4 = df3.drop_duplicates(subset=['cluster'], keep='first', inplace=False, ignore_index=False)

        # add sample name information
        df4['#sample_name'] = base

        df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')

        df5 = df4[['#sample_name', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity', 'taxonomic_annotation']]


        df5.to_csv(output_csv, sep=';', index=False, header=False)


./results/qc/barcode01/barcode01_PR2_blastn2.csv
./results/qc/barcode02/barcode02_PR2_blastn2.csv
./results/qc/barcode03/barcode03_PR2_blastn2.csv
./results/qc/barcode04/barcode04_PR2_blastn2.csv
./results/qc/barcode05/barcode05_PR2_blastn2.csv
./results/qc/barcode06/barcode06_PR2_blastn2.csv
./results/qc/barcode07/barcode07_PR2_blastn2.csv
./results/qc/barcode08/barcode08_PR2_blastn2.csv
./results/qc/barcode09/barcode09_PR2_blastn2.csv
./results/qc/barcode10/barcode10_PR2_blastn2.csv
./results/qc/barcode11/barcode11_PR2_blastn2.csv
./results/qc/barcode12/barcode12_PR2_blastn2.csv


/tmp/ipykernel_1122442/2543769090.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['#sample_name'] = base
/tmp/ipykernel_1122442/2543769090.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')
/tmp/ipykernel_1122442/2543769090.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [39]:
if os.path.exists("_PR2_eDNA.csv"):
    os.remove("_PR2_eDNA.csv")

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(file_path):
        with open(file_path, "r") as input_file, open("_PR2_eDNA.csv", "a") as output_file:
            output_file.write(input_file.read())

with open("_PR2_eDNA.csv", "r") as input_file, open("PR2_eDNA.csv", "w") as output_file:
    output_file.write("counts,cluster,accession,accession,evalue,bitscore,alignment_length,percentage_identity,taxonomic_annotation\n")
    for line in input_file:
        if not line.startswith("#"):
            output_file.write(line.replace(";", ",").replace("|", ","))

#shutil.copy("PR2_eDNA.csv", os.path.join(current_dir, "PR2_eDNA.csv"))